In [ ]:
import ee
import datetime
import pandas as pd
import time
from datetime import datetime

In [ ]:
# Authenticating and initializing the Earth Engine API
ee.Authenticate()
ee.Initialize(project = "")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
base_dir = "/content/drive/MyDrive/Data 298B Project Data/Test Dataset - Workbook 2"

In [ ]:
df = pd.read_csv(f"{base_dir}/image_metadata.csv")

In [ ]:
df.head()

,Id,Latitude,Longitude,Date and Time,Date
0,IMG_1240.JPG,22.503033,120.485581,2019:01:26 13:42:38,2019-01-26
1,IMG_1392.JPG,22.515956,120.487739,2019:01:26 15:07:09,2019-01-26
2,IMG_1326.JPG,22.501131,120.487236,2019:01:26 14:14:18,2019-01-26
3,IMG_20190308_114945.jpg,24.098481,120.669239,2019:03:08 11:49:45,2019-03-08
4,IMG_20190308_111908.jpg,24.099251,120.668691,2019:03:08 11:19:07,2019-03-08


In [ ]:
# List which contains tuples which contain the Latitude, Longitude, and Date for each row in the dataframe
coordinates_and_dates = []

for _, row in df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    date_created = row['Date']
    coordinates_and_dates.append((lat, lon, date_created))

In [ ]:
print(len(coordinates_and_dates))

20


In [ ]:
# Getting rid of duplicate tuples

from collections import OrderedDict

# Using an OrderedDict to remove duplicates while maintaining order.
unique_coordinates_and_dates_ordered = list(OrderedDict.fromkeys(coordinates_and_dates))

print(unique_coordinates_and_dates_ordered)

[(22.50303333333333, 120.48558055555556, '2019-01-26'), (22.51595555555556, 120.48773888888888, '2019-01-26'), (22.501130555555555, 120.48723611111112, '2019-01-26'), (24.09848130555556, 120.66923925, '2019-03-08'), (24.099251444444445, 120.66869083333334, '2019-03-08'), (22.501444444444445, 120.48715277777778, '2019-01-26'), (22.501113888888888, 120.48721944444443, '2019-01-26'), (22.502997222222223, 120.48503055555555, '2019-01-26'), (24.098481027777776, 120.66923794444445, '2019-03-08'), (22.51593611111111, 120.48770833333332, '2019-01-26'), (24.09872872222222, 120.66777622222224, '2019-03-08'), (22.51595, 120.4877861111111, '2019-01-26'), (24.098200527777777, 120.6693021388889, '2019-03-08'), (22.501141666666665, 120.48725833333332, '2019-01-26'), (24.074350222222225, 120.66159827777778, '2018-12-27'), (22.50290555555556, 120.485725, '2019-01-26'), (24.069349222222225, 120.65324747222223, '2018-12-27')]


In [ ]:
print(len(unique_coordinates_and_dates_ordered))

17


In [ ]:
# MODIS Satellite (latest)

import csv
import time
start_time = time.time()

def get_modis_values(latitude, longitude, date_str):
    point = ee.Geometry.Point([longitude, latitude])
    target_date = ee.Date(date_str)

    modis = ee.ImageCollection('MODIS/006/MOD13Q1')\
             .filterBounds(point)\
             .filterDate('2000-01-01', target_date)\
             .select(['NDVI', 'EVI'])\
             .sort('system:time_start', False)\
             .limit(3)  # Getting only the three most recent images

    ndvi_values = []
    evi_values = []

    for i in range(3):
        image = modis.toList(3).get(i)
        image = ee.Image(image)

        # Directly using the pre-calculated NDVI and EVI, scaled to actual values
        ndvi_val = image.select('NDVI').multiply(0.0001).reduceRegion(ee.Reducer.first(), point, 250).get('NDVI').getInfo()
        evi_val = image.select('EVI').multiply(0.0001).reduceRegion(ee.Reducer.first(), point, 250).get('EVI').getInfo()

        ndvi_values.append(ndvi_val)
        evi_values.append(evi_val)

    # Padding missing values for NDVI and EVI if fewer than 3 images are available
    while len(ndvi_values) < 3:
        ndvi_values.append(None)
    while len(evi_values) < 3:
        evi_values.append(None)

    # Combining NDVI and EVI values for the CSV
    values_list = [latitude, longitude, date_str] + ndvi_values + evi_values

    return values_list

def write_to_csv(filename, data):
    headers = ["Latitude", "Longitude", "Date",
               "NDVI MODIS", "NDVI - 1 MODIS", "NDVI - 2 MODIS",
               "EVI MODIS", "EVI - 1 MODIS", "EVI - 2 MODIS"]

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in data:
            writer.writerow(row)


# Fetching MODIS values for each location and date
results = [get_modis_values(lat, lon, date) for lat, lon, date in unique_coordinates_and_dates_ordered]

csv_file_path = f"{base_dir}/remote_sensing_data.csv"
write_to_csv(csv_file_path, results)

print(f"CSV file created: {csv_file_path}")

elapsed_time = time.time() - start_time
print(f"It took {elapsed_time:.4f} seconds to obtain the remote sensing data from the image metadata.")

CSV file created: /content/drive/MyDrive/Data 298B Project Data/Test Dataset - Workbook 2/remote_sensing_data.csv
It took 31.9716 seconds to obtain the remote sensing data from the image metadata.
